<a href="https://colab.research.google.com/github/krishaaaaaaa/python100daysofcode/blob/main/Pyspark.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


In [2]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://archive.apache.org/dist/spark/spark-3.1.1/spark-3.1.1-bin-hadoop3.2.tgz
!tar xf spark-3.1.1-bin-hadoop3.2.tgz
!pip install -q findspark
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.1-bin-hadoop3.2"
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()
spark

In [13]:
d = pd.read_csv('..//content//original (1).csv')
d.head()

,id,first_name,last_name,gender,City,JobTitle,Salary,Latitude,Longitude
0,1,Melinde,Shilburne,Female,Nowa Ruda,Assistant Professor,$57438.18,50.577407,16.496718
1,2,Kimberly,Von Welden,Female,Bulgan,Programmer II,$62846.60,48.823157,103.521820
2,3,Alvera,Di Boldi,Female,NaN,NaN,$57576.52,39.994746,116.339772
3,4,Shannon,O'Griffin,Male,Divnomorskoye,Budget/Accounting Analyst II,$61489.23,44.504721,38.130017
4,5,Sherwood,Macieja,Male,Mytishchi,VP Sales,$63863.09,NaN,37.648995


In [11]:
from pyspark.sql.functions import *

In [17]:
df = spark.read.format('csv').option('header',True).load('original (1).csv')
df.head(5)

[Row(id='1', first_name='Melinde', last_name='Shilburne', gender='Female', City='Nowa Ruda', JobTitle='Assistant Professor', Salary='$57438.18', Latitude='50.5774075', Longitude='16.4967184'),
 Row(id='2', first_name='Kimberly', last_name='Von Welden', gender='Female', City='Bulgan', JobTitle='Programmer II', Salary='$62846.60', Latitude='48.8231572', Longitude='103.5218199'),
 Row(id='3', first_name='Alvera', last_name='Di Boldi', gender='Female', City=None, JobTitle=None, Salary='$57576.52', Latitude='39.9947462', Longitude='116.3397725'),
 Row(id='4', first_name='Shannon', last_name="O'Griffin", gender='Male', City='Divnomorskoye', JobTitle='Budget/Accounting Analyst II', Salary='$61489.23', Latitude='44.5047212', Longitude='38.1300171'),
 Row(id='5', first_name='Sherwood', last_name='Macieja', gender='Male', City='Mytishchi', JobTitle='VP Sales', Salary='$63863.09', Latitude=None, Longitude='37.6489954')]

In [19]:
df.show()

+---+----------+----------+------+---------------+--------------------+---------+----------+-----------+
| id|first_name| last_name|gender|           City|            JobTitle|   Salary|  Latitude|  Longitude|
+---+----------+----------+------+---------------+--------------------+---------+----------+-----------+
|  1|   Melinde| Shilburne|Female|      Nowa Ruda| Assistant Professor|$57438.18|50.5774075| 16.4967184|
|  2|  Kimberly|Von Welden|Female|         Bulgan|       Programmer II|$62846.60|48.8231572|103.5218199|
|  3|    Alvera|  Di Boldi|Female|           null|                null|$57576.52|39.9947462|116.3397725|
|  4|   Shannon| O'Griffin|  Male|  Divnomorskoye|Budget/Accounting...|$61489.23|44.5047212| 38.1300171|
|  5|  Sherwood|   Macieja|  Male|      Mytishchi|            VP Sales|$63863.09|      null| 37.6489954|
|  6|     Maris|      Folk|Female|Kinsealy-Drinan|      Civil Engineer|$30101.16|53.4266145| -6.1644997|
|  7|     Masha|    Divers|Female|         Dachun|     

In [30]:
data = df.withColumn('Clean_City', when(df.City.isNull(),'Unknown').otherwise(df.City))
data.show()

+---+----------+----------+------+---------------+--------------------+---------+----------+-----------+---------------+
| id|first_name| last_name|gender|           City|            JobTitle|   Salary|  Latitude|  Longitude|     Clean_City|
+---+----------+----------+------+---------------+--------------------+---------+----------+-----------+---------------+
|  1|   Melinde| Shilburne|Female|      Nowa Ruda| Assistant Professor|$57438.18|50.5774075| 16.4967184|      Nowa Ruda|
|  2|  Kimberly|Von Welden|Female|         Bulgan|       Programmer II|$62846.60|48.8231572|103.5218199|         Bulgan|
|  3|    Alvera|  Di Boldi|Female|           null|                null|$57576.52|39.9947462|116.3397725|        Unknown|
|  4|   Shannon| O'Griffin|  Male|  Divnomorskoye|Budget/Accounting...|$61489.23|44.5047212| 38.1300171|  Divnomorskoye|
|  5|  Sherwood|   Macieja|  Male|      Mytishchi|            VP Sales|$63863.09|      null| 37.6489954|      Mytishchi|
|  6|     Maris|      Folk|Femal

In [26]:
data.filter(data.JobTitle.isNull()).show()

+---+----------+---------+------+------+--------+---------+----------+-----------+----------+
| id|first_name|last_name|gender|  City|JobTitle|   Salary|  Latitude|  Longitude|Clean_City|
+---+----------+---------+------+------+--------+---------+----------+-----------+----------+
|  3|    Alvera| Di Boldi|Female|  null|    null|$57576.52|39.9947462|116.3397725|   Unknown|
|  7|     Masha|   Divers|Female|Dachun|    null|$25090.87| 24.879416| 118.930111|    Dachun|
+---+----------+---------+------+------+--------+---------+----------+-----------+----------+



In [32]:
df1 = data.withColumn('Salary',data.Salary.substr(2,100).cast('float'))
df1.show()

+---+----------+----------+------+---------------+--------------------+--------+----------+-----------+---------------+
| id|first_name| last_name|gender|           City|            JobTitle|  Salary|  Latitude|  Longitude|     Clean_City|
+---+----------+----------+------+---------------+--------------------+--------+----------+-----------+---------------+
|  1|   Melinde| Shilburne|Female|      Nowa Ruda| Assistant Professor|57438.18|50.5774075| 16.4967184|      Nowa Ruda|
|  2|  Kimberly|Von Welden|Female|         Bulgan|       Programmer II| 62846.6|48.8231572|103.5218199|         Bulgan|
|  3|    Alvera|  Di Boldi|Female|           null|                null|57576.52|39.9947462|116.3397725|        Unknown|
|  4|   Shannon| O'Griffin|  Male|  Divnomorskoye|Budget/Accounting...|61489.23|44.5047212| 38.1300171|  Divnomorskoye|
|  5|  Sherwood|   Macieja|  Male|      Mytishchi|            VP Sales|63863.09|      null| 37.6489954|      Mytishchi|
|  6|     Maris|      Folk|Female|Kinsea